# Imola Programma 2019

## E se Babbo Natale esistesse, e fosse un chatbot?

* **Speaker:** Gabriele Corni [ Gabriele_Corni@iprel.it ]
* **Azienda:** IPREL Progetti S.r.l.
* **Data:** 30 marzo 2019
* **GitHub repository:** https://github.com/IprelProgetti/ImolaProgramma2019.git


### <u>Indice</u>

#### <i>Pratica: costruiamo (e proviamo) un chatbot Telegram in 5 minuti</i>

[1) Configurare il file di log e funzioni di utilità](#log)

[2) Definire il comportamento del chatbot](#comportamento)

[3) Creare il chatbot su Telegram](#creazione)

[4) Collegare il chatbot via token](#collegamento)

[5) Effettuare i bindings](#binding)

[6) Proviamolo insieme!](#test)


### Pratica: costruiamo (e proviamo) un chatbot Telegram in 5 minuti

#### 1) Configurare il file di log e funzioni di utilità<a name="log"></a>

Abbiamo paragonato un chatbot ad un server web. Ebbene, così come accade per gli accessi ad un sito internet, è parimenti importante loggare gli accessi e le richieste al chatbot.

Questo si rivelerà utile per rilevare malfunzionamenti, monitorare il traffico e soprattutto per analizzare le abitudini di utilizzo degli utenti:

* quali funzioni sono più richieste? 
* quali non vengono quasi mai invocate? 
* cosa potrebbe aumentare il valore percepito dagli utenti sulla base del loro utilizzo?
* ...

In [ ]:
import logging
import random

In [ ]:
logging.basicConfig(
    level=logging.DEBUG,
    format='[ %(asctime)s ] - -  %(levelname)s: %(name)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    filename='chatbot.log',
    filemode='w'
)
console = logging.StreamHandler()
console.setLevel(logging.INFO)
logging.getLogger('').addHandler(console)

#### 2) Definire il comportamento del chatbot<a name="comportamento"></a>

Il comportamento dei chatbot può essere definito tramite l'implementazione di semplici funzioni.  
Ogni metodo può modificare lo stato interno del bot/del suo backend e/o ritornare un messaggio all'utente relativamente all'esito dell'operazione richiesta.

Nel caso di chatbot Telegram implementato in Python, i parametri di ingresso sono generalmente due (entrambi dizionari/oggetti json):

* `bot`: istanza del bot che risponde
    * bot_id
    * bot_username
    * comandi
* `update`: i dettagli dell'evento che l'utente ha scatenato interagendo col bot
    * update_id
    * message
        * message_id
        * date
        * chat
            * id
            * type
            * username
            * first_name
            * last_name
        * from
            * id
            * username
            * first_name
            * last_name
            * is_bot
            * language_code
        * text
        * entities
        * caption_entities
        * photo
        * ...
        


###### Modello linguistico

Importa e usa qui tutte le tue librerie di NLP più avanzate :)  
noi ne simuleremo una super semplice, utile a capire il meccanismo di funzionamento

In [ ]:
modello_linguistico = {
    # da ricercare nel testo (modello linguistico "naive")
    "interiezioni"          : ["ciao", "buongiorno"],
    "parole_chiave_azienda" : ["iprel"],
    
    # calcolo risposta    
    "info_azienda" : ["_IPREL Progetti_ è una società di engineering operante nell'area di controllo del processo in diversi settori produttivi: ceramico, imballaggio, alimentare, chimico e meccanico.",
                      "L'azienda nasce nel 1995 da una joint venture tra *AEPI Industrie*, azienda leader nello scenario nazionale e mondiale dell'automazione industriale, ed il suo maggiore cliente, *SACMI Imola*.",
                      "L'azienda è fortemente orientata all'*innovazione tecnologica*, e si adopera per il continuo miglioramento dei processi produttivi, utilizzando le più moderne apparecchiature reperibili sul mercato mondiale.",
                      "Per tutte le altre info, visita http://www.iprel.it/\n"
                     ],
    
    "messaggio_benvenuto" : "Benvenuto a Imola Programma 2019",
    "messaggio_errore"    : "Mi dispiace, non ti ho capito.\nPotresti ripetere per favore?\nGrazie",
    "messaggio_aiuto"     : "Chiedimi pure informazioni in linguaggio naturale, o clicca sul pulsante `/` per usufruire dei comandi preimpostati.",
    "messaggio_funzioni"  : "Sono pronto per salutarti e parlarti di IPREL Progetti!\nSe vuoi, lasciaci `/feedback`!",
    "messaggio_feedback"  : "Vuoi *votare* (1-10), lasciarci un *feedback testuale* o farci una *domanda*?"
}

###### Funzioni comportamentali

Definisci qui il comportamento del chatbot Telegram.

In [ ]:
def welcome(bot, update):
    """La funzione con cui dare il benvenuto all'utente"""
    nome_utente = update.message.chat.first_name
    
    logging.info('{} ha avviato una chat'.format(
        nome_utente
    ))
    
    msg = "*Benvenuto, {nome}!*\n{funzioni}".format(
        nome=nome_utente,
        funzioni=modello_linguistico["messaggio_funzioni"]
    )
    
    bot.send_message(
        chat_id=update.message.chat_id,
        text=msg,
        parse_mode=ParseMode.MARKDOWN,
        reply_markup=ReplyKeyboardRemove()
    )

In [ ]:
def help_me(bot, update):
    """La funzione con cui spiegare all'utente cosa può fare"""
    nome_utente = update.message.chat.first_name
    
    logging.info('{} ha chiesto le info di utilizzo'.format(
        nome_utente
    ))
    
    msg = modello_linguistico["messaggio_aiuto"]
    
    bot.send_message(
        chat_id=update.message.chat_id,
        text=msg,
        parse_mode=ParseMode.MARKDOWN,
        reply_markup=ReplyKeyboardRemove()
    )

In [ ]:
voti = ["{}".format(i) for i in range(1, 11)]
parere = "FEEDBACK"
domanda = "DOMANDA"

In [ ]:
def build_keyboard(buttons, n_cols, header_buttons=None, footer_buttons=None):
    """La funzione con cui creare una griglia di bottoni per le risposte preimpostate"""
    menu = [buttons[i:i + n_cols] for i in range(0, len(buttons), n_cols)]
    if header_buttons:
        menu.insert(0, header_buttons)
    if footer_buttons:
        menu.append(footer_buttons)
    return menu

In [ ]:
def feedback(bot, update, user_data):
    """La funzione con cui consentire all'utente di dare un voto al talk, fare domande o lasciare feedback"""
    nome_utente = update.message.chat.first_name
    user_data["fbk"] = True
    
    logging.info('{} sta lasciando un feedback'.format(
        nome_utente
    ))
    
    griglia_voto = build_keyboard(voti, n_cols=5, footer_buttons=[parere, domanda])
    msg = modello_linguistico["messaggio_feedback"]
    
    bot.send_message(
        chat_id=update.message.chat_id,
        text=msg,
        parse_mode=ParseMode.MARKDOWN,
        reply_markup=ReplyKeyboardMarkup(griglia_voto)
    )

In [ ]:
def error(bot, update, error):
    """La funzione che gestisce le eventuali situazioni di errore"""
    nome_utente = update.message.chat.first_name
    
    logging.warning("{} ha causato l'errore {}".format(
        nome_utente,
        error
    ))
    
    msg = modello_linguistico["messaggio_errore"]
    
    bot.send_message(
        chat_id=update.message.chat_id,
        text=msg,
        parse_mode=ParseMode.MARKDOWN,
        reply_markup=ReplyKeyboardRemove()
    )

In [ ]:
def dialogo_con_utente(bot, update, user_data):
    def invia_saluto(nome):
        logging.info('{} ha salutato'.format(
            nome
        ))
        
        messaggio_saluto = "{saluto} {nome}!\n{msg}".format(
            saluto=random.choice(modello_linguistico["interiezioni"]),
            nome=nome,
            msg=modello_linguistico["messaggio_benvenuto"]
        )
        return invia(messaggio_saluto)
    
    def invia_info_azienda(nome):
        logging.info('{} ha chiesto di Iprel'.format(
            nome
        ))
        
        messaggio_azienda = "\n\n".join(modello_linguistico["info_azienda"])
        return invia(messaggio_azienda)
    
    def invia_risposta_generica(nome, testo):
        if user_data["fbk"]:
            user_data["fbk"] = False
            
            log_msg = "{nome} ha detto: {testo}".format(
                nome=nome,
                testo=testo
            )
            risposta = "Grazie per aver partecipato, {}!".format(nome)
        
        else:
            log_msg = '{nome} ha chiesto qualcosa che non è stato capito: "{testo}"'.format(
                nome=nome,
                testo=testo
            )
            risposta = modello_linguistico["messaggio_errore"]
        
        logging.info(log_msg)   
        messaggio_risposta = risposta
        return invia(messaggio_risposta)
    
    def voto_utente(nome, testo):
        logging.info('{nome} ha votato: "{testo}"'.format(
            nome=nome,
            testo=testo
        ))
            
        messaggio_ringraziamento = "Grazie per aver votato, {}!".format(nome)
        user_data["fbk"] = False
        return invia(messaggio_ringraziamento)
    
    def parere_utente(nome, testo):
        logging.info('{nome} sta scrivendo un parere sul talk'.format(
            nome=nome
        ))
            
        messaggio_ringraziamento = "Cosa ti è piaciuto, cosa non hai capito, cosa miglioreresti?\nIl talk è stato chiaro?\nIl talk è stato interessante?"
        return invia(messaggio_ringraziamento)
    
    def domanda_utente(nome, testo):
        logging.info('{nome} sta scrivendo una domanda'.format(
            nome=nome
        ))
            
        messaggio_ringraziamento = "Cosa vorresti chiedere?\nChe curiosità hai?"
        return invia(messaggio_ringraziamento)
    
    def invia(msg):
        bot.send_message(
            chat_id=update.message.chat_id,
            text=msg,
            parse_mode=ParseMode.MARKDOWN,
            reply_markup=ReplyKeyboardRemove()
        )

    nome_utente = update.message.chat.first_name
    testo_immesso = update.message.text.lower().strip()
    if "fbk" not in user_data:
        user_data["fbk"] = False
        
    if any([stringa in testo_immesso for stringa in modello_linguistico["interiezioni"]]) and not user_data["fbk"]:
        return invia_saluto(nome_utente)
    elif any([stringa in testo_immesso for stringa in modello_linguistico["parole_chiave_azienda"]]) and not user_data["fbk"]:
        return invia_info_azienda(nome_utente)
    elif testo_immesso in voti:
        return voto_utente(nome_utente, testo_immesso)
    elif testo_immesso == parere.lower():
        return parere_utente(nome_utente, testo_immesso)
    elif testo_immesso == domanda.lower():
        return domanda_utente(nome_utente, testo_immesso)
    else:
        return invia_risposta_generica(nome_utente, testo_immesso)

#### 3) Creare il chatbot su Telegram<a name="creazione"></a>

La procedura di creazione va effettuata tramite [`@BotFather`](https://telegram.me/botfather) via app `Telegram`.

**Solo lo sviluppatore** dovrà preoccuparsi di questa fase.

Si utilizzano i seguenti parametri per la creazione del bot:

##### first_name:

```
IprelAtImolaProgramma2019
```

##### username: 

```
IprelAtImolaProgramma2019_bot
```

##### comandi:

```
help - chiedimi come usarmi
feedback - lascia un tuo parere
```

Utilizzare il `token` restituito da [`@BotFather`](https://telegram.me/botfather) per collegare l'istanza del chatbot ai comportamenti programmati via software.

#### 4) Collegare il chatbot via token<a name="collegamento"></a>

Il token viene copiato in chiaro nel sorgente per motivi di tempo e per fini dimostrativi.

Sarebbe più opportuno salvare il token come variabile d'ambiente e caricarne il valore trasparentemente.

In caso di applicazioni reali è caldamente consigliata l'adozione di adeguate misure di sicurezza.

In [ ]:
TELEGRAM_BOT_TOKEN = '805021961:AAFoEKHBgQJ4mHAaoQV20qZXeJcsvnQWmwQ'

In [ ]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from telegram import ParseMode, ReplyKeyboardRemove, ReplyKeyboardMarkup

In [ ]:
# Istanziare l'EventHandler del chatbot creato passandogli il codice identificativo.
updater = Updater(TELEGRAM_BOT_TOKEN)

In [ ]:
# Ottenere il dispatcher di eventi.
dp = updater.dispatcher

#### 5) Effettuare i bindings<a name="binding"></a>

L'ultima cosa che resta da fare è associare ogni comando del chatbot ad una delle funzioni precedentemente definite.

In [ ]:
# Associare al dispatcher i comportamenti del chatbot:
# # Comandi (risposte a necessità utente)
dp.add_handler(CommandHandler("start", welcome))
dp.add_handler(CommandHandler("help", help_me))
dp.add_handler(CommandHandler("feedback", feedback, pass_user_data=True))

# # Comprensione del linguaggio naturale (dialogo con utente)
dp.add_handler(MessageHandler(Filters.text, dialogo_con_utente, pass_user_data=True))

# # Errori (notificare errori nell'utilizzo o occorsi)
dp.add_error_handler(error)

#### 6) Proviamolo insieme!<a name="test"></a>

**Lato Python:** avviare e mantenere attivo il chatbot finchè il processo non viene interrotto

In [ ]:
logging.info('Bot avviato')

updater.start_polling()
updater.idle()

logging.info('Bot spento')

**Lato Telegram:** aprire l'app, cercare dalla lente di ingrandimento l'utente [`IprelAtImolaProgramma2019`](https://telegram.me/IprelAtImolaProgramma2019_bot) ed avviare una sessione di utilizzo premendo il pulsante `START` in basso.

* Il bot accoglierà ogni nuovo utente con il messaggio di `welcome()`
* I comandi configurati saranno accessibili dall'apposito tasto `/` di fianco alla chat, e gestiti dagli appositi `CommandHandler`
* L'interazione in linguaggio naturale verrà gestita dalla logica di `dialogo_con_utente()`
* Eventuali errori verranno gestiti e notificati dal metodo di `error()`